In [1]:
import os
import pandas as pd
import numpy as np
import json
import argparse

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import permutation_test
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
from statsmodels.stats import multitest

from tfsemb_class import tsne, save_pickle, add_speech
from tfsplt_encoding import get_cmap_smap, aggregate_data, organize_data
from tfsplt_utils import read_folder, load_pickle, get_con_color
from tfsplt_brainmap import get_sigelecs, Colorbar, make_brainmap
from tfsplt_brainmap_cat import make_brainmap_cat

## Data Loading

In [2]:
max_cor_0 = pd.read_csv("../data/plotting/paper-mia/AA_gpt-neo-125M_elec_max_corr.csv", index_col=0)
max_cor_1 = pd.read_csv("../data/plotting/paper-mia/AA_gpt-neo-1.3B_elec_max_corr.csv", index_col=0)
max_cor_2 = pd.read_csv("../data/plotting/paper-mia/AA_gpt-neo-2.7B_elec_max_corr.csv", index_col=0)
max_cor_3 = pd.read_csv("../data/plotting/paper-mia/AA_gpt-neox-20b_elec_max_corr.csv", index_col=0)

In [9]:
LAYERS_DICT = {
    "first": [0,0,0,0],
    "mid": [6,12,16,22],
    "last": [12,24,32,44],
    "best": [10,22,18,14]
}

## Best Layer Max Corr

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast.svg"

args = Args()

# args.color_split = [Colorbar(colorscale="hot",bar_min=0,bar_max=0.4,title="max encoding performance (r)")]
args.color_split = [Colorbar(colorscale="viridis",bar_min=0,bar_max=0.4,title="max encoding performance (r)")]

# Brain Map Plots
model_num = 3
model_df = max_cor_3.copy()
model_df["electrode"] =  model_df.elec_name.str.replace("_comp","")
model_df["subject"] = model_df.electrode.str.split("_", n=1, expand=True)[0]
model_df["effect"] =  model_df.max_corr

df_plot = model_df.loc[model_df.layer == LAYERS_DICT["best"][model_num], ("subject","electrode", "effect")]
fig = make_brainmap(args, df_plot, args.outfile) # plot png

## Best Layer Max Corr Diff

In [10]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast-%s.svg"

args = Args()

# Colorsplit
pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "rgb(255,248,240)"], [1, "rgb(255,0,0)"]],bar_min = 0,bar_max=0.15)
neg_bar = Colorbar(title="Δ corr neg",colorscale=[[0, "rgb(0,0,255)"], [1, "rgb(240,248,255)"]],bar_min=-0.15,bar_max=0)
args.color_split = [neg_bar,0,pos_bar]

# Brain Map Plots
for layer_type in LAYERS_DICT:
  if layer_type != "best":
    continue
  print(layer_type)
  # calculate difference
  model_0 = max_cor_0.loc[max_cor_0.layer == LAYERS_DICT[layer_type][0],("elec_name","max_corr")]
  model_1 = max_cor_1.loc[max_cor_1.layer == LAYERS_DICT[layer_type][1],("elec_name","max_corr")]
  model_2 = max_cor_2.loc[max_cor_2.layer == LAYERS_DICT[layer_type][2],("elec_name","max_corr")]
  model_3 = max_cor_3.loc[max_cor_3.layer == LAYERS_DICT[layer_type][3],("elec_name","max_corr")]
  model_2.sort_values(by="elec_name",inplace=True)
  model_3.sort_values(by="elec_name",inplace=True)
  model_3 = model_3.assign(max_corr2 = model_1.max_corr)
  model_3 = model_3.assign(effect = model_3.max_corr - model_3.max_corr2)
  # format datum
  model_3["electrode"] =  model_3.elec_name.str.replace("_comp","")
  model_3["subject"] = model_3.electrode.str.split("_", n=1, expand=True)[0]

  df_plot = model_3.loc[:, ("subject","electrode", "effect")]
  fig = make_brainmap(args, df_plot, args.outfile % layer_type) # plot png

SyntaxError: invalid syntax (365761592.py, line 20)

## Max Corr per Layer

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast-%s.png"

args = Args()

args.color_split = [Colorbar(colorscale="viridis",bar_max=0.45)]

# Brain Map Plots
model_num = 3
model_df = max_cor_4.copy()
model_df["electrode"] =  model_df.elec_name.str.replace("_comp","")
model_df["subject"] = model_df.electrode.str.split("_", n=1, expand=True)[0]
model_df["effect"] =  model_df.max_corr

for layer in np.arange(LAYERS_DICT["first"][model_num],LAYERS_DICT["last"][model_num] + 1):
  print(layer)
  df_plot = model_df.loc[model_df.layer == layer, ("subject","electrode", "effect")]
  fig = make_brainmap(args, df_plot, args.outfile % f"{layer:02}") # plot png

## Max Corr

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast.png"

args = Args()

args.color_split = [Colorbar(title="best layer",colorscale="viridis",bar_max=0.45, bar_min=0)]

df_plot = max_cor_1.sort_values(["max_corr"], ascending=False).groupby("elec_name").first()
df_plot.reset_index(inplace=True)
df_plot["effect"] = df_plot.max_corr
df_plot["electrode"] =  df_plot.elec_name.str.replace("_comp","")
df_plot["subject"] = df_plot.electrode.str.split("_", n=1, expand=True)[0]
df_plot = df_plot.loc[:, ("subject","electrode", "effect")]
fig = make_brainmap(args, df_plot, args.outfile) # plot png

## Max Corr Diff

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast-best-diff.png"

args = Args()

# Colorsplit
pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "rgb(255,248,240)"], [1, "rgb(255,0,0)"]],bar_max=0.1)
neg_bar = Colorbar(title="Δ corr neg",colorscale=[[0, "rgb(0,0,255)"], [1, "rgb(240,248,255)"]],bar_min=-0.1)
args.color_split = [neg_bar,0,pos_bar]


df_plot = max_cor_3.sort_values(["max_corr"], ascending=False).groupby("elec_name").first()
df_plot.reset_index(inplace=True)
df_plot2 = max_cor_2.sort_values(["max_corr"], ascending=False).groupby("elec_name").first()
df_plot2.reset_index(inplace=True)
df_plot["max_corr2"] = df_plot2.max_corr
df_plot["effect"] = df_plot.max_corr - df_plot.max_corr2
df_plot["electrode"] =  df_plot.elec_name.str.replace("_comp","")
df_plot["subject"] = df_plot.electrode.str.split("_", n=1, expand=True)[0]

df_plot = df_plot.loc[:, ("subject","electrode", "effect")]
fig = make_brainmap(args, df_plot, args.outfile) # plot png

## Max Corr Layer

In [22]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast.svg"

args = Args()

args.color_split = [Colorbar(title="Layer with best correlation (relative %)",colorscale="agsunset",bar_max=1, bar_min=0)]
# args.color_split = [Colorbar(title="Layer with best correlation (relative %)",colorscale="inferno",bar_max=1, bar_min=0)]

model_num = 3
df_plot = max_cor_3.sort_values(["max_corr"], ascending=False).groupby("elec_name").first()
df_plot.reset_index(inplace=True)
min = LAYERS_DICT["first"][model_num]
max = LAYERS_DICT["last"][model_num]
df_plot["effect"] = df_plot.layer / (max - min)
# df_plot["effect"] = df_plot.layer
df_plot["electrode"] =  df_plot.elec_name.str.replace("_comp","")
df_plot["subject"] = df_plot.electrode.str.split("_", n=1, expand=True)[0]
df_plot = df_plot.loc[:, ("subject","electrode", "effect")]
fig = make_brainmap(args, df_plot, args.outfile) # plot png

Number of electrodes for encoding: 160
Number of electrodes for plotting: 160
Plot Hemisphere
Plotting 160 Electrodes with ave coordinates
Writing to ../podcast.svg


## Area brainmap

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast.png"

args = Args()
color_list = ["rgb(55,126,184)","rgb(255,127,0)","rgb(77,175,74)","rgb(152,78,163)"]
args.colors = color_list

area_file = "../data/plotting/paper-mia/area_electrodes.json"
with open(area_file, "r") as j:
    area_dict = json.loads(j.read())

area_df = []
for area in area_dict:
    for elec in area_dict[area]:
        area_df.append([elec, area])

area_df = pd.DataFrame(area_df)
area_df.columns = ["electrode","effect"]
area_df["subject"] = area_df.electrode.str.split("_", n=1, expand=True)[0]
fig = make_brainmap_cat(args, area_df, args.outfile) # plot png
